## Import libraries

In [22]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

## Data location

In [23]:
data_loc = "S://Docs//Personal//MAEVE//Data//"

## Pull data

In [24]:
path = data_loc + "BTC_price_1h.csv"
df = pd.read_csv(path)

print(f"Data shape: {df.shape}")

print(f"Date range: {df.Datetime.min()} - {df.Datetime.max()}")

Data shape: (17017, 7)
Date range: 2021-02-01 00:00:00+00:00 - 2023-01-20 13:00:00+00:00


## Custom functions

In [25]:
def calc_MA(df, timeperiod):
    df[f'MA{timeperiod}'] = df['Close'].rolling(window=timeperiod).mean()
    return df

In [ ]:
def log_trade(price, pos, cash, sats, init_cash=100):

    df = pd.DataFrame()

    df['price'] = [price]
    df['tradeType'] = [pos]
    df['cash'] = [cash]
    df['sats'] = [sats]
    df['profit/loss'] = np.where(df['sats'] > 0,
                                 (df['sats']*df['price']) - init_cash, df['cash'] - init_cash)

    return df


## Identify test periods

In [ ]:
# Bull market
# Bear market
# Black swan events: Covid, FTX, Luna/3AC
# Bullish news: Tesla buy in, Futures ETF approval
# Accumulation/ flat
# Low volume time periods

## Calculate moving averages

In [26]:
# Calculate MA
MALst = [12, 20, 24, 30, 40, 48, 50, 60, 100, 200]

for MA in MALst:
    df = calc_MA(df, MA)

df.shape


(17017, 17)

In [29]:
# Initialize the strategy variables
current_position = None  # "buy" or "sell"
cash = 100.00  # Starting cash
sats = 0  # Starting BTC

trades_df = pd.DataFrame()

# Iterate over the rows of the dataframe
for index, row in df.iterrows():
    # Calculate the MA20 and MA40
    MA20 = row['MA20']
    MA40 = row['MA40']

    # Check if the MA20 is higher than the MA40
    if MA20 > MA40:
        # If we're not currently holding any BTC, buy BTC
        if current_position != "buy":
            sats = round(cash / row['Close'], 8)
            cash = 0
            current_position = "buy"
            trades_df = pd.concat([trades_df, log_trade(row['Close'], current_position, cash, sats)])
            
    # Check if the MA20 is lower than the MA40
    elif MA20 < MA40:
        # If we're currently holding BTC, sell
        if current_position == "buy":
            cash = round(sats * row['Close'], 2)
            sats = 0
            current_position = "sell"
            trades_df = pd.concat([trades_df, log_trade(row['Close'], current_position, cash, sats)])

# Print the final cash and sats value
print("Final cash value:", cash)
print("Final sats value:", sats)


Final cash value: 0
Final sats value: 0.00301445


In [30]:
trades_df

,price,tradeType,cash,sats,profit/loss
0,34678.085938,buy,0.00,0.002884,0.000156
0,37264.953125,sell,107.46,0.000000,7.460000
0,37727.167969,buy,0.00,0.002848,7.460179
0,38789.417969,sell,110.49,0.000000,10.490000
0,43241.972656,buy,0.00,0.002555,10.490159
...,...,...,...,...,...
0,17422.755859,buy,0.00,0.003048,-46.899970
0,20791.261719,sell,63.37,0.000000,-36.630000
0,20879.103516,buy,0.00,0.003035,-36.630042
0,20949.736328,sell,63.58,0.000000,-36.420000


In [31]:
trades_df['profit/loss'].max()


34.18000000000001

In [32]:
trades_df['profit/loss'].min()


-47.94007500578125